# Cheat sheet :  Netgen/NGSolve 

&ensp; La librairie python [Netgen/NGSolve (NGPy)](https://ngsolve.org/) est un mailleur/solveur pour des problèmes élements finis similaire à [FreeFem++](https://freefem.org/) ou [FEniCS](https://fenicsproject.org/).
Netgen permet de créer la géométrie et le maillage. NGSolve se concentre sur l'assemblage des matrices et la résolution du problème éléments finis. Des interfaces sont possibles avec d'autres mailleurs/solveur tels que [GMSH](https://gmsh.info/) ou [SciPy](https://scipy.org/).<br>
Ce document a pour objectif de présenter les fonctions de bases de la bibliothèque, plus de détails et exemples d'applications sont disponibles sur le site de [NGPy](https://docu.ngsolve.org/latest/).

___
## I. Netgen
Netgen fonctionne sur le système Point,Ligne,Surface (si 3D). Pour définir un maillage 2D, on définit donc un ensemble de points par leurs coordonnées puis les lignes/splines qui les relient. 

In [ ]:
from netgen.geom2d import SplineGeometry #Import du module de géométrie 2D (aussi disponible en CSG)
from ngsolve.comp import Mesh # Import de l'objet maillage de NGsolve pour la conversion maillage Netgen -> maillage NGsolve 
from ngsolve.webgui import Draw #Import de la fonction d'affichage pour les notebook jupyter.

geo = SplineGeometry() # Initialisation de l'objet qui va contenir la géometrie

# Ajouter des points 
pts_coords = [(-1, 1), (-1, -1),(1, -1), (1, 1)] # On va créer un carré centré sur (0,0) de côté 2 unités.
pts_geo = [geo.AppendPoint(pts[0], pts[1]) for pts in pts_coords] # On rajoute les points à la géometrie et on récupère leur indice pour pouvoir construire les segments

# Ajouter des segments
geo.Append(["line", pts_geo[0], pts_geo[1]], leftdomain=1, rightdomain=0, bc = 'left') # On définit ici un segment qui va du point 0 au point 1 
geo.Append(["line", pts_geo[1], pts_geo[2]], leftdomain=1, rightdomain=0, bc = 'bottom') # Chaque segment est ici une frontière entre le domaine 1 et le domaine 0 qui n'est pas maillé sur Netge,
geo.Append(["line", pts_geo[2], pts_geo[3]], leftdomain=1, rightdomain=0, bc = 'right') # Note : Il est important que chaque domaine soit fermé. 
geo.Append(["line", pts_geo[3], pts_geo[0]], leftdomain=1, rightdomain=0, bc = 'top')

geo.SetMaterial(1,'domain') # Pour faciliter l'indentification des différents domaines, ils peuvent être nommé. 

# Generation du maillage 
ngmesh = geo.GenerateMesh(maxh = 0.2) # Le maillage est généré avec une taille maximale d'élement de 0.2 unités. 
mesh = Mesh(ngmesh)
Draw(mesh)

D'autres fonctions pour générer des formes telles que des rectangles, des cercles et tracer des splines sont disponibles et détaillées dans ce [tutorial](https://docu.ngsolve.org/latest/netgen_tutorials/define_2d_geometries.html).
___
## II. NGSolve
<a id='NGSolve'></a>
## A. Resolution d'un problème éléments finis. 
Pour exemplifier cette partie, on choisit de résoudre l'equation de [Poisson](https://docu.ngsolve.org/latest/whetting_the_appetite/poisson.html) en imposant des conditions de Dirichlet sur l'entièreté du bord du carré. <br>
Le problème en forme faible s'ecrit : 
$$ \begin{cases} \underbrace{\int_\Omega \left<\nabla u,\nabla v\right> \mathop{dx}}_{\text{forme bilinéaire}} = \underbrace{\int_\Omega f v }_{\text{forme linéaire}} \quad \forall v \\  u = 0  ~\text{sur}~ \partial \Omega \end{cases}$$ 
On choisit de résoudre ce problème sur l'espace discretisé $H^1$. On définit donc cet espace et les fonctions symboliques d'essai u et de test v. On exprime ensuite les formes bilinéaires et linéaires avant de les assembler et de resoudre le problème matriciel. <br>
Pour le terme source, la fonction $f(x,y) = 32 \, x²y²$ est choisie arbitrairement

In [ ]:
from ngsolve import * # On importe le reste de NGSolve
from ngsolve.webgui import Draw # A rejouter dans la section de code jupyter si Draw n'affiche rien.

## Définition de l'espace de résolution et des fonctions d'essai et de test
fes = H1(mesh, order = 1, dirichlet = "right|top|left|bottom") # Les conditions de dirichlet permettent ainsi de savoir le nombre de degrés de libertés lors de la résolution.
# D'autres espaces telles que Hrot, 
u = fes.TrialFunction() # Fonction d'essai
v = fes.TestFunction()  # Fonction de test

## Définition de formes bilinéaires et linéaires

# Forme bilinéaire
K = BilinearForm(fes, symmetric = True)
K += grad(u)*grad(v)*dx(definedon = mesh.Materials("domain")) # On ajoute la forme bilinéaire avec l'opérateur d'intégration dx.
# ou K += SymbolicBFI(grad(u)*grad(v), definedon = mesh.Materials("domain"))

# Forme linéaire
l = LinearForm(fes)
f = CoefficientFunction(32*x*x*y*y) # On utilise ici une CoefficientFunction. Elle permet de définir une expression symbolic qui dépend des coordonnées (x,y) du noeud de maillage
l += f*v*dx(definedon=mesh.Materials("domain")) #Il possible de définir des fonctions sur des lignes en utilisant l'intégrateur ds(definedon=mesh.Boundaries("nom")). 
# ou l += SymbolicLFI(f*v,definedon=mesh.Materials("domain"))

## Assemblage et résolution
K.Assemble() #création de la matrice K
l.Assemble() #création du vecteur de terme source

# Stockage de la solution  (GridFunction)
gfu = GridFunction(fes) # on définit une Gridfunction qui stockera la valeur de la solution par noeud. 

# Resolution (Interfaçage possible avec SciPy)
gfu.vec.data = K.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * l.vec #On fait une décomposition de Cholesky de la matrice creuse K pour résoudre le problème. 

# Affichage de la solution
Draw(gfu,mesh)

## B. Affichage et fonctions auxiliaires

Après résolution du problème éléments quelques transformations peuvent être nécessaires pour rendre interprétable le résultat. 

In [ ]:
# Affichage d'une fonction vectorielle
vec_data = CoefficientFunction(grad(gfu))
Draw(vec_data,mesh,vectors = { "grid_size":20}) # L'argument vectors permets d'afficher les flèches et de définir leur nombre. 

# On peut aussi choisir de n'afficher que sur x (0) du gradient de la solution.
Draw(CoefficientFunction(grad(gfu)[0]),mesh)

# Ou d'appliquer une rotation sur le gradient et en affichant juste la norme du vecteur.
Draw(CoefficientFunction((grad(gfu)[1],-grad(gfu)[0])),mesh) 

Pour calculer certaines quantités comme le flux, la compliance ou les forces, il est nécessaire d'intégrer sur le maillage.

In [ ]:
# En passant par la formulation symbolique
forme_symbolique = gfu * dx(definedon = mesh.Materials('domain'))
print(Integrate(forme_symbolique,mesh))

# De manière plus directe
print(Integrate(gfu,mesh,definedon = mesh.Materials('domain')))